In [1]:
from collections import Counter

import numpy as np
import pandas as pd
import tensorflow.keras as keras

In [ ]:
train_df = pd.read_csv('../input/digit-recognizer/train.csv')
pred_df = pd.read_csv('../input/digit-recognizer/test.csv')
sample_submission_df = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
print(train_df.shape, pred_df.shape)

In [ ]:
X = train_df.iloc[:, 1:].values.reshape((-1, 28, 28, 1)) / 255.0
y = train_df.iloc[:, 0].values
pred_x = pred_df.values.reshape((-1, 28, 28, 1)) / 255.0
print(X.shape, y.shape, pred_x.shape)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(input_shape=X.shape[1:], kernel_size=5, filters=18, padding="same",
                        activation=keras.activations.swish),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Conv2D(kernel_size=5, filters=48, padding="valid", activation=keras.activations.swish),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Conv2D(kernel_size=5, filters=360, padding="valid", activation=keras.activations.swish),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation=keras.activations.swish),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation=keras.activations.swish),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(10, activation=keras.activations.softmax),
])
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Nadam(),
              metrics=[keras.metrics.sparse_categorical_accuracy])
model.summary()
history = model.fit(x=X,
                    y=y,
                    batch_size=32,
                    epochs=7,
                    validation_split=0.01)

In [ ]:
pred_y = np.argmax(model.predict(pred_x), axis=1)
print(Counter(pred_y))
pd.DataFrame({'ImageId': range(1, len(pred_y) + 1), 'Label': pred_y}).to_csv('submit.csv', index=False)